<a href="https://colab.research.google.com/github/Alinejj/Alinejj/blob/main/Natural_Disaster_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
import plotly.express as px
import folium
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import requests
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import copy

In [ ]:
#loading natural disaster dataset
disasters_dataset = pd.read_csv('/content/natural_disaster.csv')

In [ ]:
#print first 5 rows
disasters_dataset.head()

,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,...,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,NaN,NaN,3.221647,NaN,NaN,NaN,NaN
1,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,NaN,NaN,3.221647,NaN,NaN,NaN,NaN
2,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,NaN,NaN,NaN,NaN,25000.0,3.350513,NaN,NaN,NaN,NaN
3,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,3.350513,NaN,NaN,NaN,NaN
4,1902,10,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,3.350513,NaN,NaN,NaN,NaN


In [ ]:
#number of rows and columns
disasters_dataset.shape

(16126, 45)

In [ ]:
#statisctical measures of the data
disasters_dataset.describe()

,Year,Seq,Aid Contribution,Dis Mag Value,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
count,16126.000000,16126.000000,6.770000e+02,4.946000e+03,16126.00000,15739.000000,12498.000000,16126.000000,15418.000000,12570.000000,1.141300e+04,3.895000e+03,9.220000e+03,2.430000e+03,1.161700e+04,1.096000e+03,5.245000e+03,15811.000000
mean,1996.764790,714.784820,1.254136e+05,4.735038e+04,1996.77837,6.444374,15.233957,1996.835607,6.576728,15.775020,2.842866e+03,2.621102e+03,8.823612e+05,7.329314e+04,7.165088e+05,7.986514e+05,7.247835e+05,63.215103
std,20.159065,1929.635089,2.997875e+06,3.094242e+05,20.15571,3.393965,8.953821,20.143010,3.352965,8.865486,6.860595e+04,3.440343e+04,8.573913e+06,5.230058e+05,7.718598e+06,3.057638e+06,4.723131e+06,26.734285
min,1900.000000,1.000000,1.000000e+00,-5.700000e+01,1900.00000,1.000000,1.000000,1900.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,3.400000e+01,2.000000e+00,3.221647
25%,1989.000000,93.000000,1.750000e+02,7.000000e+00,1989.00000,4.000000,7.000000,1989.000000,4.000000,8.000000,6.000000e+00,1.400000e+01,1.244750e+03,5.725000e+02,6.500000e+02,5.000000e+04,8.300000e+03,45.692897
50%,2001.000000,270.000000,7.210000e+02,1.515000e+02,2001.00000,7.000000,15.000000,2001.000000,7.000000,16.000000,2.000000e+01,5.000000e+01,1.000000e+04,3.000000e+03,5.965000e+03,1.725000e+05,6.000000e+04,68.415379
75%,2011.000000,486.000000,3.511000e+03,1.129650e+04,2011.00000,9.000000,23.000000,2011.000000,9.000000,23.000000,6.300000e+01,2.000000e+02,9.182300e+04,1.750000e+04,5.825500e+04,5.000000e+05,3.173000e+05,84.252733
max,2021.000000,9881.000000,7.800000e+07,1.302587e+07,2021.00000,12.000000,31.000000,2021.000000,12.000000,31.000000,3.700000e+06,1.800000e+06,3.300000e+08,1.585000e+07,3.300000e+08,6.000000e+07,2.100000e+08,100.000000


In [ ]:
disasters_dataset['total Deaths'].value_counts

<bound method IndexOpsMixin.value_counts of 0        9002
1        9001
2          12
3           3
4          10
         ... 
16121     449
16122      75
16123     599
16124      20
16125     481
Name: Seq, Length: 16126, dtype: int64>

In [ ]:
non_numerical_cols = disasters_dataset.select_dtypes(include=["object"]).columns
print(f"Non-numerical columns: {non_numerical_cols}")

Non-numerical columns: Index(['Glide', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype', 'Disaster Subsubtype', 'Event Name', 'Country',
       'ISO', 'Region', 'Continent', 'Location', 'Origin', 'Associated Dis',
       'Associated Dis2', 'OFDA Response', 'Appeal', 'Declaration',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')


In [ ]:
disasters_dataset.drop(non_numerical_cols, axis=1, inplace=True)

In [ ]:
imputer = SimpleImputer(strategy="mean")
data_imputed = imputer.fit_transform(disasters_dataset)

In [ ]:
# Update the non_numerical_cols variable after dropping the non-numerical columns
non_numerical_cols = disasters_dataset.select_dtypes(include=["object"]).columns

# Iterate through the updated non_numerical_cols
for col in non_numerical_cols:
    disasters_dataset[col] = le.fit_transform(disasters_dataset[col])

In [ ]:
imputer = KNNImputer(n_neighbors=5)
data_imputed = imputer.fit_transform(disasters_dataset)

In [ ]:
fig = px.histogram (disasters_dataset, x = 'Total Deaths', color = 'End Year', nbins = 30, title = 'Total Death of citizens by the End Year')
fig.show()

In [ ]:
model = XGBClassifier(n_estimator = 100, learning_rate = 0.1, max_depth = 5)

In [ ]:
time_step = 10
num_features = 10
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, num_features)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
X = disasters_dataset.drop(columns = 'Total Deaths', axis = 1)
Y = disasters_dataset['Total Deaths']

In [ ]:
print(X)

       Year   Seq  Aid Contribution  Dis Mag Value  Start Year  Start Month  \
0      1900  9002               NaN            NaN        1900          NaN   
1      1900  9001               NaN            NaN        1900          NaN   
2      1902    12               NaN            8.0        1902          4.0   
3      1902     3               NaN            NaN        1902          4.0   
4      1902    10               NaN            NaN        1902         10.0   
...     ...   ...               ...            ...         ...          ...   
16120  2021   270               NaN            NaN        2021          5.0   
16121  2021   449               NaN            NaN        2021          7.0   
16122  2021    75               NaN            NaN        2021          2.0   
16123  2021   599               NaN            NaN        2021          9.0   
16125  2021   481               NaN            NaN        2021          5.0   

       Start Day  End Year  End Month  End Day  No 

In [ ]:
print(Y)

0          11000.0
1        1250000.0
2           2000.0
3           1000.0
4           6000.0
           ...    
16120         13.0
16121         11.0
16122         31.0
16123        131.0
16125          7.0
Name: Total Deaths, Length: 11413, dtype: float64


In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X)

StandardScaler()

In [ ]:
standarized_data = scaler.transform(X)

In [ ]:
print(standarized_data)

[[-4.55053619 11.96774756         nan ...         nan         nan
  -2.18371205]
 [-4.55053619 11.96636713         nan ...         nan         nan
  -2.18371205]
 [-4.45601967 -0.44238753         nan ...         nan -0.15408569
  -2.17900996]
 ...
 [ 1.16771342 -0.35541996         nan ...         nan -0.14469917
          nan]
 [ 1.16771342  0.36792941         nan ...         nan         nan
          nan]
 [ 1.16771342  0.20503776         nan ...         nan         nan
          nan]]


In [ ]:
X = standarized_data
Y = disasters_dataset['Total Deaths']

In [ ]:
print(X)
print(Y)

[[-4.55053619 11.96774756         nan ...         nan         nan
  -2.18371205]
 [-4.55053619 11.96636713         nan ...         nan         nan
  -2.18371205]
 [-4.45601967 -0.44238753         nan ...         nan -0.15408569
  -2.17900996]
 ...
 [ 1.16771342 -0.35541996         nan ...         nan -0.14469917
          nan]
 [ 1.16771342  0.36792941         nan ...         nan         nan
          nan]
 [ 1.16771342  0.20503776         nan ...         nan         nan
          nan]]
0          11000.0
1        1250000.0
2           2000.0
3           1000.0
4           6000.0
           ...    
16120         13.0
16121         11.0
16122         31.0
16123        131.0
16125          7.0
Name: Total Deaths, Length: 11413, dtype: float64


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(11413, 17) (9130, 17) (2283, 17)


In [ ]:
classifier = svm.SVC(kernel = 'linear')

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [100]:
# Train the classifier
classifier.fit(X_train, Y_train)

# Predict the labels for the training data
X_train_prediction = classifier.predict(X_train)

# Calculate the accuracy of the training data predictions
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [102]:
print('Accuracy score of the data: ', training_data_accuracy)

Accuracy score of the data:  0.08521358159912376


In [110]:

input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

# Create a copy of the input data
input_data_with_extra_features = copy.deepcopy(input_data)

# Add dummy features to the input data
input_data_with_extra_features = list(input_data_with_extra_features) + [0] * 9

# Reshape the input data
input_data_reshaped = np.asarray(input_data_with_extra_features).reshape(1, -1)

# Create a new StandardScaler object
new_scaler = StandardScaler()

# Fit the new_scaler object on the input data
new_scaler.fit(input_data_reshaped)

# Standardize the input data using the new StandardScaler object
std_data = new_scaler.transform(input_data_reshaped)

prediction = classifier.predict(std_data)
print(prediction)

# Interpret the prediction
if (prediction[0] == 0):
    print('The natural disaster is unlikely to occur.')
else:
    print('The natural disaster is likely to occur.')


[1.]
The natural disaster is likely to occur.
